In [ ]:
!pip install stellargraph[demos]

In [ ]:
!pip install networkx

In [ ]:
#!export PYTHONPATH=$PYTHONPATH:~/.local/lib/python3.7/site-packages

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 500)
import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler
from sklearn.model_selection import train_test_split

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score

from stellargraph import globalvar

from stellargraph import datasets
from IPython.display import display, HTML

## DATASET PROCESING

Загружаем граф:

In [2]:
graph = pd.read_csv("graph_omsk_F_encoded.csv")

In [3]:
graph 

,Unnamed: 0,edge_id,length,width,rm_car_speed,type,light_begin,light_end,turn_to,payments_flags,is_city_edge,barrier,turn_restrictions,traverse,passability,pedo_offset,bad_road,def_speed,road_exit,adjacent,road_classes_1,road_classes_2,road_classes_3,road_classes_4,road_classes_5,road_classes_6,road_classes_7,road_classes_8,road_classes_9,sm_classes_0,sm_classes_1,sm_classes_2,sm_classes_3,sm_classes_4,sm_classes_5,sm_classes_6,sm_classes_7,sm_classes_8,sm_classes_9,sm_classes_10,sm_classes_11,sm_classes_12,sm_classes_13,sm_classes_14,sm_classes_15,sm_classes_16,sm_classes_17,sm_classes_18,sm_classes_19,sm_classes_20,sm_classes_21,sm_classes_22,sm_classes_23,sm_classes_24,sm_classes_25,sm_classes_26,sm_classes_27,sm_classes_28,sm_classes_29,sm_classes_30,sm_classes_31,sm_classes_32,sm_classes_33,sm_classes_34,sm_classes_35,sm_classes_36,sm_classes_37,sm_classes_38,sm_classes_39,sm_classes_40,sm_classes_41,sm_classes_42,sm_classes_43,sm_classes_44,sm_classes_45,sm_classes_46,sm_classes_47,sm_classes_48,style_<pandas.io.formats.style.Styler object at 0x000001BA9BC98AC0>,lanes_0,lanes_1,lanes_2,lanes_3,lanes_4,lanes_5,rm_car_classes_0,rm_car_classes_1,rm_car_classes_2,rm_car_classes_3,rm_car_classes_4,rm_car_classes_5,rm_car_classes_6,rm_car_classes_7,rm_car_classes_8,rm_car_classes_9,rm_car_classes_10,rm_car_classes_11,rm_car_classes_12,rm_car_classes_13,rm_car_classes_14,rm_car_classes_15,rm_car_classes_16,rm_car_classes_17,rm_car_classes_18
0,0,564384472498178,242,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"1, 2, 111230, 3, 111231",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,564384472498179,242,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"0, 111308, 111309",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2,564384472498182,224,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"3, 6, 29, 7, 28",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,3,564384472498183,224,4,60,0,0,0,1,0,1,0,0,1,1,1,0,60,0,"1, 2, 111230, 0, 111231",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,564384472498184,444,4,90,0,0,0,1,0,0,0,0,1,1,1,0,90,0,"5, 16, 17",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231683,231683,212117746323998779,7,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231682,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231684,231684,212117746366813904,11,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231685,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231685,231685,212117746366813905,11,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231684,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
231686,231686,212117746395564138,16,3,3,0,0,0,0,0,0,0,0,0,0,0,0,3,0,231687,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Генерируем edge list:

In [4]:
def get_index_tuples(df):
    lst = []
    for i in tqdm(range(len(df))):
        tmp = df['adjacent'][i].split(',')
        for edge in tmp:
            lst.append([i, int(edge)])
            
    return lst
lst = get_index_tuples(graph)
lst = pd.DataFrame(lst)
lst

100%|██████████| 231688/231688 [00:03<00:00, 62467.63it/s]


,0,1
0,0,1
1,0,2
2,0,111230
3,0,3
4,0,111231
...,...,...
1149487,231683,231682
1149488,231684,231685
1149489,231685,231684
1149490,231686,231687


In [5]:
edge_data = pd.DataFrame(
    {
        "source": list(lst[0]),
        "target": list(lst[1])
    }
)
edge_data

,source,target
0,0,1
1,0,2
2,0,111230
3,0,3
4,0,111231
...,...,...
1149487,231683,231682
1149488,231684,231685
1149489,231685,231684
1149490,231686,231687


Дропаем лишнее (структурное описание/копии индекса/артефакты): 

In [6]:
node_data = graph.drop(["edge_id", "adjacent", "Unnamed: 0", "style_<pandas.io.formats.style.Styler object at 0x000001BA9BC98AC0>"], axis = 1)

Создаем граф:

In [7]:
G = sg.StellarGraph(
    {"street": node_data}, {"connection": edge_data}
)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 231688, Edges: 1149492

 Node types:
  street: [231688]
    Features: float32 vector, length 100
    Edge types: street-connection->street

 Edge types:
    street-connection->street: [1149492]
        Weights: all 1 (default)
        Features: none


## GRAPHSAGE EMBEDDINGS

In [9]:
nodes = list(G.nodes())
number_of_walks = 2
length = 5

In [10]:
unsupervised_samples = UnsupervisedSampler(
    G, nodes=nodes, length=length, number_of_walks=number_of_walks
)

In [11]:
batch_size = 50
epochs = 5
num_samples = [10, 5]

In [12]:
generator = GraphSAGELinkGenerator(G, batch_size, num_samples)
train_gen = generator.flow(unsupervised_samples)

In [13]:
layer_sizes = [50, 50]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.0, normalize="l2"
)

In [14]:
x_inp, x_out = graphsage.in_out_tensors()

In [15]:
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [16]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(run_eagerly=True,
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [20]:
history = model.fit(
    train_gen,
    epochs=1,
    verbose=1,
    use_multiprocessing=True,
    workers=22,
    shuffle=True,
)

74141/74141 [==============================] - 2798s 38ms/step - loss: 0.5503 - binary_accuracy: 0.7635


Получаем эмбеддинги вершин:

In [18]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from stellargraph.mapper import GraphSAGENodeGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_ids = [i for i in range(len((graph)))]
node_gen = GraphSAGENodeGenerator(G, batch_size, num_samples).flow(node_ids)

node_embeddings = embedding_model.predict(node_gen, workers=4, verbose=1)

4634/4634 [==============================] - 99s 21ms/step


In [19]:
pd.DataFrame(node_embeddings).to_csv("emb_omsk.csv")

In [22]:
node_embeddings = pd.read_csv("emb_omsk.csv", index_col = 0).values.tolist()

In [23]:
len(node_embeddings)

231688

## CONCATENATE EMBEDDINGS

Загружаем файл с путями:

In [35]:
routes = pd.read_csv("abakan_full_routes_final_weather_L_NaN_filtered.csv", index_col = 0)

Дропаем лишние копии индекса (если они имеются):

In [39]:
c_routes = routes.drop(["index", "level_0","Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1"], axis = 1)
#c_routes = routes.drop(["Unnamed: 0.1","Unnamed: 0.1.1"], axis = 1)

In [40]:
c_routes

,dist_to_b,dist_to_a,start_timestamp,RTA,real_dist,pred_dist,rebuildCount,route_type,start_point_meters,finish_point_meters,start_point_part,finish_point_part,edges,time,speed,length,day_period,week_period,clouds,snow,temperature,wind,pressure
0,65,2,2020-12-14 16:00:22.0,757,2936,3495,2,traffic,1.0,25.0,0.785760,0.531041,"' 19423207912140509', ' 19423207912140505', ' ...","' 3360', ' 66600', ' 33720', ' 10500', ' 1700'...","' 15', ' 15', ' 15', ' 36', ' 36', ' 27', ' 26...","' 14', ' 90', ' 78', ' 55', ' 17', ' 147', ' 5...",2,0,1,0,-6,180,746
1,48,0,2020-12-12 10:11:27.0,501,4245,4163,2,traffic,6.0,16.0,0.722993,0.290994,"' 19423207912141719', ' 19423207912141691', ' ...","' 4800', ' 23160', ' 2640', ' 11840', ' 2700',...","' 15', ' 15', ' 15', ' 20', ' 20', ' 35', ' 40...","' 20', ' 34', ' 11', ' 38', ' 15', ' 62', ' 13...",1,0,1,0,-9,180,750
2,28,55,2020-12-06 12:21:17.0,372,3523,3514,0,traffic,5.0,25.0,0.554582,0.899702,"' 19423207912137899', ' 19423207912137895', ' ...","' 3120', ' 21000', ' 26351', ' 11793', ' 10303...","' 15', ' 15', ' 29', ' 29', ' 29', ' 29', ' 35...","' 13', ' 25', ' 172', ' 95', ' 83', ' 25', ' 1...",2,0,1,0,-21,315,752
3,81,2,2020-12-13 14:25:20.0,475,3787,4178,1,traffic,11.0,32.0,0.496820,0.434923,"' 19423207912142002', ' 19423207912142014', ' ...","' 8640', ' 8880', ' 15800', ' 960', ' 6080', '...","' 15', ' 15', ' 15', ' 15', ' 50', ' 45', ' 20...","' 36', ' 37', ' 45', ' 4', ' 15', ' 52', ' 90'...",2,0,5,2,-7,180,746
4,17,0,2020-12-04 17:43:41.0,643,4595,4672,0,traffic,1.0,10.0,0.566270,0.321134,"' 140060152264356559', ' 140060152264357499', ...","' 3840', ' 5280', ' 4080', ' 13400', ' 19926',...","' 15', ' 15', ' 15', ' 15', ' 19', ' 17', ' 17...","' 16', ' 22', ' 17', ' 35', ' 26', ' 35', ' 29...",2,0,3,0,-18,270,749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122360,72,0,2020-12-29 10:02:33.0,371,781,765,1,traffic,1.0,17.0,0.901433,0.920319,"' 19423207912137925', ' 19423207912126325', ' ...","' 960', ' 15675', ' 18822', ' 9154', ' 9360', ...","' 15', ' 29', ' 35', ' 35', ' 35', ' 15', ' 15...","' 4', ' 86', ' 183', ' 89', ' 91', ' 78', ' 90...",1,0,1,0,-26,0,758
122361,76,2,2020-12-25 16:17:11.0,329,1165,1191,2,traffic,41.0,71.0,0.429539,0.634195,"' 19423207912132093', ' 19423207912132309', ' ...","' 6640', ' 15840', ' 1440', ' 1728', ' 981', '...","' 45', ' 35', ' 20', ' 50', ' 33', ' 33', ' 40...","' 83', ' 154', ' 8', ' 24', ' 9', ' 157', ' 30...",2,0,1,0,-18,270,744
122362,54,18,2020-12-18 16:52:58.0,916,6542,6570,0,traffic,8.0,20.0,0.830191,0.582137,"' 140060152380920943', ' 19423207912122553', '...","' 2454', ' 3432', ' 12436', ' 39600', ' 7200',...","' 44', ' 43', ' 33', ' 45', ' 44', ' 49', ' 41...","' 30', ' 41', ' 114', ' 495', ' 88', ' 93', ' ...",2,0,1,0,-2,180,746
122363,30,0,2020-12-27 14:53:40.0,187,1145,1263,3,traffic,7.0,40.0,0.590148,0.460256,"' 19423207912140481', ' 19423207912140477', ' ...","' 6000', ' 10280', ' 6928', ' 385', ' 5528', '...","' 15', ' 15', ' 28', ' 28', ' 28', ' 28', ' 28...","' 25', ' 22', ' 15', ' 3', ' 43', ' 46', ' 54'...",2,1,1,0,-30,0,762


Дропаем лишние признаки:

In [41]:
X_pre = c_routes.drop(["real_dist","pred_dist","rebuildCount", "speed", "time", "length", "route_type","start_timestamp"], axis = 1) 

Проверяем индексацию:

In [64]:
X_pre = X_pre.reset_index().drop(["index"], axis = 1)

In [42]:
import multiprocessing as mp
import psutil

In [65]:
X_pre.loc[3298, "edges"].replace("'", "")

' 19423207912142760,  19423207912142762,  19423207912142764,  19423207912120461,  19423207912120429,  140060152389075915,  19423207912120373,  19423207912135755,  19423207912120207,  19423207912120217,  19423207912147277,  19423207912147275,  19423207912147273,  19423207912120421,  19423207912120453,  19423207912139667,  19423207912139665,  19423207912120615,  19423207912120665,  19423207912120685,  19423207912120753,  19423207912137859,  140060152309421295,  19423207912120933,  19423207912120975,  19423207912121001,  19423207912121045,  19423207912121133,  19423207912121183,  19423207912121185,  19423207912121097,  19423207912121249,  19423207912121261,  19423207912121262,  19423207912121100'

Процессим edges: (будет исправлено позднее)

In [ ]:
for i in tqdm(range(len(X_pre.loc[:, "edges"]))):
    X_pre.loc[i, "edges"] = X_pre.loc[i, "edges"].replace("'", "")



  0%|          | 0/122311 [00:00<?, ?it/s]

  0%|          | 17/122311 [00:00<12:03, 169.06it/s]

  0%|          | 36/122311 [00:00<11:41, 174.38it/s]

  0%|          | 56/122311 [00:00<11:22, 179.13it/s]

  0%|          | 76/122311 [00:00<11:10, 182.39it/s]

  0%|          | 95/122311 [00:00<11:06, 183.29it/s]

  0%|          | 114/122311 [00:00<11:06, 183.28it/s]

  0%|          | 133/122311 [00:00<11:00, 184.96it/s]

  0%|          | 153/122311 [00:00<10:45, 189.11it/s]

  0%|          | 173/122311 [00:00<10:37, 191.52it/s]

  0%|          | 193/122311 [00:01<10:31, 193.44it/s]

  0%|          | 213/122311 [00:01<10:25, 195.28it/s]

  0%|          | 233/122311 [00:01<10:22, 195.97it/s]

  0%|          | 253/122311 [00:01<10:26, 194.88it/s]

  0%|          | 273/122311 [00:01<10:26, 194.71it/s]

  0%|          | 293/122311 [00:01<10:30, 193.61it/s]

  0%|          | 313/122311 [00:01<10:32, 192.74it/s]

  0%|          | 333/122311 [00:01<10:37, 191.47it/s]

  0%|          | 353/122

In [33]:
def summ(first, second):
    return [x + y for x, y in zip(first, second)]

In [34]:
def zerolistmaker(n):
    return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]

Размер эмбеддинга:

In [ ]:
emb_len = len(node_embeddings[0])

Определяем размер батча для распараллеливания построения эмбеддинга подграфа: 

In [ ]:
workers_count = 20

In [36]:
part_const = int(len(X_pre)/workers_count)
part_const

In [38]:
def spawn():
    dataframe_list = []
    procs = list()
    n_cpus = psutil.cpu_count()
    manager = mp.Manager()
    return_list = manager.list()
    
    for cpu in range(workers_count):
        d = dict(border = part_const*(cpu+1))
        p = mp.Process(target=run_child, args=(d, return_list))
        p.start()
        procs.append(p)
    for p in procs:
        p.join()
        print('joined')
    
    return return_list

def run_child(border, return_list):
    emb = []
    for i in tqdm(range(border['border'] - part_const, border['border'])):
        path_emb = zerolistmaker(emb_len)
#        print(X_pre.loc[i, "edges"])
        arr = X_pre.loc[i, "edges"].replace("'", "").split(",")
        for j in range(len(arr)):
            ind = graph[graph["edge_id"] == int(arr[j])].index
            if (len(ind) != 0):
                ind = graph[graph["edge_id"] == int(arr[j])].index[0]
            else:
                print(str(i) + " " + str(j) + " " + arr[j] + " "+ str(graph[graph["edge_id"] == int(arr[j])].index[0]))
            path_emb = summ(node_embeddings[ind], path_emb)
        emb.append(path_emb)
    return_list.append([(int(border['border'])-part_const)/part_const, emb])
 #   print(border['border'])
#    df = pd.DataFrame(dataframe[(int(border['border'])-part_const):border['border']]).reset_index()
#    return_list.append([(int(border['border'])-part_const)/part_const,processing.flatternize(16, filepath = "", dataframe = df)])

In [47]:
a = spawn()

  3%|▎         | 967/38654 [00:34<22:41, 27.68it/s] 
Process Process-81:
Traceback (most recent call last):
  File "/home/user/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/user/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-38-15ab38f2351a>", line 27, in run_child
    path_emb = summ(node_embeddings[graph[graph["edge_id"] == int(arr[j])].index[0]], path_emb)
  File "/home/user/conda/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4297, in __getitem__
    return getitem(key)
  8%|▊         | 3025/38654 [01:51<21:51, 27.16it/s]  
Process Process-71:
Traceback (most recent call last):
  File "/home/user/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/user/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipytho

joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined
joined


Собираем данные:

In [48]:
normal_container_s = []
for i in tqdm(range(len(a))):
    normal_container_s.append(a[i])
a_ = sorted(normal_container_s,key=lambda x: (x[0],x[1]))
a_c = a_.copy() 

100%|██████████| 15/15 [00:05<00:00,  2.81it/s]


In [41]:
a_ = sorted(normal_container_s,key=lambda x: (x[0],x[1]))

In [40]:
result = a_[0][1]
for i in range(1, len(a_)):
    result += a_[i][1]

Технические проверки:

In [41]:
X_pre_c = X_pre.drop(X_pre[X_pre["snow"] == -1].index)

In [42]:
set(X_pre_c["day_period"])

{0, 1, 2, 3}

Объединяем данные путей с эмбеддингами, завершаем подготовку данных к обучению:

In [43]:
X = X_pre_c.join(pd.DataFrame(result)).dropna().drop(["edges"], axis = 1).reset_index()

In [44]:
wind_dir_classes = pd.get_dummies(X.wind, prefix='wind_dir_class')
day_classes = pd.get_dummies(X.day_period, prefix='day_class')
Y = X["RTA"]
X = X.join(wind_dir_classes).join(day_classes).drop(["wind", "day_period", "index", "RTA"], axis = 1)

In [237]:
X

,dist_to_b,dist_to_a,start_point_meters,finish_point_meters,start_point_part,finish_point_part,week_period,clouds,snow,temperature,pressure,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,wind_dir_class_0,wind_dir_class_45,wind_dir_class_90,wind_dir_class_135,wind_dir_class_180,wind_dir_class_225,wind_dir_class_270,wind_dir_class_315,day_class_0,day_class_1,day_class_2,day_class_3
0,65,2,1.0,25.0,0.785760,0.531041,0,1,0,-6,746,-0.376006,0.010730,-0.521707,0.529505,0.072768,-0.570181,0.893035,0.178625,-0.131129,0.593840,-1.739916,2.162847,-0.192242,-0.212529,-0.115392,-0.746198,0.142935,0.844580,-0.536325,0.140521,0.549880,-0.782209,-1.452595,-0.341372,-0.348165,-4.995228,7.663724,7.257641,5.196574,-7.990702,-1.789182,1.343504,1.616370,-2.951402,-4.850649,9.713345,1.231133,11.372359,-9.660663,-11.193359,-4.520683,3.490075,10.740411,2.466341,3.946379,11.285643,-3.718355,-2.983879,7.029447,-8.135970,0,0,0,0,1,0,0,0,0,0,1,0
1,48,0,6.0,16.0,0.722993,0.290994,0,1,0,-9,750,-0.362883,0.908755,0.489060,0.774793,-0.564856,0.770183,0.412008,0.079918,-0.095636,-0.881027,-0.176162,1.279454,-0.092752,0.289835,-0.404084,0.920207,-0.090695,0.230744,-1.302206,0.121754,-0.420945,-1.776792,0.195273,-1.119397,-1.463135,-7.533638,6.371610,5.495108,17.896498,-2.582500,-3.852495,-6.771907,1.286739,0.913801,-11.505458,4.364692,3.409313,11.998533,0.591081,3.317276,-1.195859,9.055771,7.189842,2.289729,-1.361499,2.937896,-10.841299,-1.274217,4.748329,-2.926804,0,0,0,0,1,0,0,0,0,1,0,0
2,28,55,5.0,25.0,0.554582,0.899702,0,1,0,-21,752,-0.693340,0.489173,0.002343,-0.035548,0.304810,-0.453584,0.394266,-0.188830,-0.241907,0.094928,-1.273565,1.095714,0.278704,0.147429,-0.318472,0.448826,-0.365054,-0.155066,-0.772260,0.320581,0.710872,-1.228038,0.151877,-0.425174,-0.103549,-4.539829,8.301054,7.277459,11.310374,-6.418864,-4.792228,-5.033050,1.477466,-2.198175,-7.612733,6.511021,2.692419,11.891631,-3.980692,-2.665986,-4.543228,7.435151,6.422519,2.902728,0.424909,3.936236,-9.266490,-2.454951,4.968851,-4.338659,0,0,0,0,0,0,0,1,0,0,1,0
3,81,2,11.0,32.0,0.496820,0.434923,0,5,2,-7,746,-0.418066,0.804341,0.288086,0.828555,-0.582304,1.040349,0.992729,0.591425,-0.208818,-0.511893,-0.332175,2.286261,0.106721,0.106716,-0.265854,0.233786,0.014271,1.310453,-1.935296,0.183934,-0.613048,-1.632913,-0.655518,-1.387138,-1.674256,-10.520425,1.208943,2.834036,10.295662,0.384023,7.278770,7.255364,5.072306,-0.127730,-5.988675,6.938361,4.440772,7.762119,-3.261579,-4.311776,2.760305,-0.346503,13.743379,-2.296564,4.507484,12.491275,2.610795,1.272329,4.727434,-5.198290,0,0,0,0,1,0,0,0,0,0,1,0
4,17,0,1.0,10.0,0.566270,0.321134,0,3,0,-18,749,-0.112733,-0.592023,-0.359273,1.041017,0.026790,0.099414,0.951379,1.216984,-0.493290,0.905835,-2.824047,2.328193,0.008542,-0.092805,-0.244953,-1.260505,0.020443,1.263685,-0.510743,0.815501,1.014011,-0.796308,-1.741219,-0.537862,-0.300831,-5.956315,8.287328,6.532058,4.079544,-8.992649,5.960517,9.365645,2.267140,-2.282911,-4.474512,11.701925,5.595899,9.028473,-11.780375,-15.779933,-4.946520,-1.558060,15.749723,0.872403,8.493009,16.936642,3.108961,-2.535924,7.123422,-8.930993,0,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117800,44,0,4.0,16.0,0.688172,0.663870,0,5,2,-7,746,-0.009488,-0.479401,-0.147627,-0.273627,0.410974,-0.283429,0.060610,0.179343,-0.119082,0.754953,-1.017337,-0.027606,0.249803,-0.089034,0.111611,-0.524155,-0.197032,0.144654,0.264998,0.332786,0.614578,0.605336,-0.230374,0.277798,0.700540,2.353035,0.138683,-1.098573,-1.627357,1.340564,4.169949,2.043796,3.087656,-2.051396,0.226637,0.193985,2.924710,0.183323,-1.537579,-2.033706,-0.398465,-3.130929,1.415919,-0.790248,3.379712,5.18307

Модель #1:

In [46]:
from sklearn.linear_model import ElasticNetCV
model = ElasticNetCV()

In [49]:
prediction = model.fit(X.iloc[0:100000, :], list(Y[0:100000]))

In [51]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
a_RMSE = mean_squared_error(Y[100000:110000], prediction.predict(X.iloc[100000:110000, :]))
a_MAE = mean_absolute_error(Y[100000:110000], prediction.predict(X.iloc[100000:110000, :]))
pd.DataFrame([[a_RMSE, a_MAE]], index = ["Abakan"], columns = ["RMSE", "MAE"])

,RMSE,MAE
Abakan,110971.797885,128.499699


Модель #2:

In [52]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [65]:
modelNN = tf.keras.models.Sequential()
modelNN.add(tf.keras.Input(shape=(73,)))
modelNN.add(tf.keras.layers.Dense(128, activation='relu'))
modelNN.add(tf.keras.layers.Dense(128, activation='relu'))
modelNN.add(tf.keras.layers.Dense(1))

In [66]:
modelNN.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [67]:
train_x = np.asarray(X.iloc[0:100000, 0:73])
train_y = np.asarray(Y[0:100000])

In [68]:
test_x = np.asarray(X.iloc[100000:110000, 0:78])
test_y = np.asarray(Y[100000:110000])

In [69]:
from  tensorflow.keras.utils import normalize, to_categorical
train_x = normalize(train_x, axis=1)
test_x =  normalize(test_x, axis=1)

In [70]:
historyNN = modelNN.fit(train_x, train_y, epochs = 30)

Epoch 1/30
3125/3125 [==============================] - 3s 892us/step - loss: 134.7035
Epoch 2/30
3125/3125 [==============================] - 3s 873us/step - loss: 114.0983
Epoch 3/30
3125/3125 [==============================] - 3s 878us/step - loss: 112.3875
Epoch 4/30
3125/3125 [==============================] - 3s 843us/step - loss: 112.7516
Epoch 5/30
3125/3125 [==============================] - 3s 867us/step - loss: 111.0957
Epoch 6/30
3125/3125 [==============================] - 3s 861us/step - loss: 113.8426
Epoch 7/30
3125/3125 [==============================] - 3s 876us/step - loss: 110.7969
Epoch 8/30
3125/3125 [==============================] - 3s 864us/step - loss: 109.9296
Epoch 9/30
3125/3125 [==============================] - 3s 885us/step - loss: 109.5350
Epoch 10/30
3125/3125 [==============================] - 3s 870us/step - loss: 111.1274
Epoch 11/30
3125/3125 [==============================] - 3s 870us/step - loss: 107.7334
Epoch 12/30
3125/3125 [==================

In [71]:
result = modelNN.evaluate(
    test_x,
    test_y, verbose=0)

In [72]:
result

107.6177749633789